In [1]:
from bestbuy import *
from credentials import *
from ebaysdk.finding import Connection as finding
import ebaysdk
import numpy as np
import pandas as pd
import multiprocessing

In [2]:
bb = BestbuyClient(api_key)

# columns for price table
upcs_for_price_frame = []
ebay_prices = []
best_buy_prices = []

# columns for match table
ebay_names_for_match_frame = []
best_buy_names_for_match_frame = []
match_inds = []

# columns for best buy table
upcs_for_best_buy_frame = []
names_for_best_buy_frame = []

# columns for ebay table
upcs_for_ebay_frame = []
names_for_ebay_frame = []
ids_for_ebay_frame = []


bb.products_fields = ['upc', 'name','regularPrice']

In [ ]:
def processPage(pageNbr):
    try:
        products = bb.products(page = pageNbr)
        for product in products:
            best_buy_name = product['name']
            product_upc  = product['upc']
            best_buy_price = product['regularPrice']
            # add to best buy table
            upcs_for_best_buy_frame.append(product_upc)
            names_for_best_buy_frame.append(best_buy_name)
            f = finding()
            f.execute('findItemsByProduct', '<productId type="UPC">' + product_upc + '</productId>')
            dom = f.response_dom()
            items = dom.getElementsByTagName('item')
            if len(items) > 0:
                try:
                    ebay_names = list(map(lambda x : x.getElementsByTagName('title')[0].firstChild.nodeValue, items))
                    first_item = items[0]
                    ebay_id = first_item.getElementsByTagName('productId')[0].firstChild.nodeValue
                    ebay_price = first_item.getElementsByTagName('sellingStatus')[0].getElementsByTagName('currentPrice')[0].firstChild.nodeValue

                    # add to price table
                    upcs_for_price_frame.append(product_upc)
                    ebay_prices.append(ebay_price)
                    best_buy_prices.append(best_buy_price)

                    # add to ebay table
                    for ebay_name in ebay_names:
                        upcs_for_ebay_frame.append(product_upc)
                        names_for_ebay_frame.append(ebay_name)
                        ids_for_ebay_frame.append(ebay_id)
                        # add to match table
                        ebay_names_for_match_frame.append(ebay_name)
                        best_buy_names_for_match_frame.append(best_buy_name)
                        match_inds.append(1)
                except:
                    pass
            else:
                ebay_id = 0
            try:
                f = finding()
                f.execute('findItemsAdvanced', {'keywords': best_buy_name})
                dom = f.response_dom()
                items = dom.getElementsByTagName('item')[:15]
                if len(items) > 0:
                    for item in items:
                        try:
                            curr_ebay_id = item.getElementsByTagName('productId')[0].firstChild.nodeValue
                        except:
                            curr_ebay_id = -1
                        if curr_ebay_id != ebay_id:
                            curr_ebay_name = item.getElementsByTagName('title')[0].firstChild.nodeValue
                            best_buy_names_for_match_frame.append(best_buy_name)
                            ebay_names_for_match_frame.append(curr_ebay_name)
                            match_inds.append(0)
            except:
                pass
    except:
        pass

In [ ]:
for i in np.arange(1,500):
    processPage(i)

ERROR:ebaysdk:findItemsAdvanced: Internal Server Error, Domain: CoreRuntime, Severity: Error, errorId: 2030, Internal error: [com.ctc.wstx.exc.WstxLazyException] com.ctc.wstx.exc.WstxUnexpectedCharException: Unexpected character ' ' (code 32); expected a semi-colon after the reference for entity 'B'
 at javax.xml.stream.SerializableLocation@41de41de


In [ ]:
price_frame = pd.DataFrame({'UPC': upcs_for_price_frame, 'best_buy_price': best_buy_prices
    , 'ebay_price': ebay_prices})
ebay_frame = pd.DataFrame({'UPC': upcs_for_ebay_frame, 'product_id': ids_for_ebay_frame
    , 'name': names_for_ebay_frame})
best_buy_frame = pd.DataFrame({'UPC': upcs_for_best_buy_frame, 'name': names_for_best_buy_frame})
match_frame = pd.DataFrame({'best_buy_name': best_buy_names_for_match_frame
    , 'ebay_name': ebay_names_for_match_frame, 'match_ind': match_inds})

In [ ]:
# price_frame.to_csv('ebay_best_buy_data/price_table_3.csv', index=False, encoding='utf-8')
# ebay_frame.to_csv('ebay_best_buy_data/ebay_table_3.csv', index=False, encoding='utf-8')
# best_buy_frame.to_csv('ebay_best_buy_data/best_buy_table_3.csv', index=False, encoding='utf-8')
# match_frame.to_csv('ebay_best_buy_data/match_table_3.csv', index=False, encoding='utf-8')

In [8]:
match_frame.loc[match_frame.match_ind==0]

,best_buy_name,ebay_name,match_ind
0,! (Bonus Track) (Japan) - CD,FREE-LIVE! +7-JAPAN SHM-CD BONUS TRACK D50,0
1,! (Bonus Track) (Japan) - CD,2015 JUSTIN BIEBER PURPOSE with bonus track (T...,0
2,! (Bonus Track) (Japan) - CD,NAKED END GAME with One Bonus Track (Total 13...,0
3,! (Bonus Track) (Japan) - CD,2015 JAPAN BRIAN SETZER ORCHESTRA ROCKIN' RUDO...,0
4,! (Bonus Track) (Japan) - CD,JEFFREY FOSKETT-CLASSIC HARMONY-JAPAN CD BONUS...,0
5,! (Bonus Track) (Japan) - CD,JEFF LYNNE-ARMCHAIR THEATRE-JAPAN MINI LP SHM-...,0
6,! (Bonus Track) (Japan) - CD,BRIAN SETZER-ROCKIN`RUDOLPH-JAPAN CD BONUS TRA...,0
7,! (Bonus Track) (Japan) - CD,KHYMERA The Grand Design with One Bonus Track ...,0
8,! (Bonus Track) (Japan) - CD,KYLIE MINOGUE-KYLIE CHRISTMAS-JAPAN CD BONUS T...,0
9,! (Bonus Track) (Japan) - CD,JEFF LYNNE'S ELO ALONE IN THE UNIVERSE Bonus T...,0


In [16]:
best_buy_frame.loc[list(map(lambda x : 'Agatha Christie\'s Poirot: Series 2' in x, best_buy_frame.name))]
# match_frame.loc[match_frame.best_buy_name=='All the Right Reasons - CD']

,UPC,name
27335,054961873791,Agatha Christie's Poirot: Series 2 [2 Discs] [...
27336,054961873692,Agatha Christie's Poirot: Series 2 [3 Discs] (...
